<style type="text/css">

div.h2 {
    background-color: steelblue; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 24px; 
    max-width: 1500px; 
    margin-top: 50px;
    margin-bottom:4px;
}
    
body {
  font-size: 12px;
}    
     
                                    
                                      
div.h3 {
    color: #159957; 
    font-size: 18px; 
    margin-top: 20px; 
    margin-bottom:4px;
    text-decoration: underline;
}
   
                                      
div.h1 {
    background-color: steelblue; 
    color: white;
    padding: 8px; 
    padding-right: 300px; 
    font-size: 32px; 
    margin-top: 20px; 
    max-width: 1500px;
    margin-bottom: 8px;
}    
</style>    
 

<div class='h1'> Analysis for US Police</div>

Policing is a tiresome job and a job which requires much attention too. I will be trying to act like a analyst for Police chief and help uncover interesting mysteries about brave policemen who are martyred while they serve their duty. Let's see what I can uncover


> I wished to upload a image, yet Kaggle is throwing me 1MB source errors

In [ ]:
!pip install bar_chart_race
!pip install folium
!pip install -U nlp_profiler

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bar_chart_race as bcr
import folium

from nlp_profiler.core import apply_text_profiling

In [ ]:
home = '../input/police-violence-in-the-us'
df_police = pd.read_csv(os.path.join(home, 'police_deaths_538.csv'))
# df_people = pd.read_csv(os.path.join(home, 'police_killings.csv'))

state_geo = '../input/usa-states-geojson/us-states.json'

df_ml = pd.read_csv("../input/tester/sample.csv", sep="\t")

In [ ]:
! java --version

In [ ]:
df_police.head().T

The police_deaths dataset contains the follownig columns:

- person: A string with position and name of Police officers who were killed on line of duty
- dept: The department person belonged to
- eow: It refers to the date of death of a police officer who was killed in the line of duty.
- cause: It refers to the cause by which the police officer was killed on and mention which place it happend etc.
- cause_short: Abrreviated cause of why a person was killed
- date: date in YY-MM-DD format
- canine: Whether a dog was killed or not?
- dept_name: Which department the person who died belongs to?
- state: state where person belongs to?

# Quick look at data

### Cause why police where martyered

In [ ]:
cause_count = df_police.cause_short.value_counts()[:4]
cause_count.plot(kind='pie', fontsize=15, autopct='%1.1f%%')
plt.ylabel('')
plt.title("Top 5 causes of why police people are martyered")

### States where fatalities are more

In [ ]:
state_count = df_police.state.value_counts()[:5]
state_count.plot(kind='pie', fontsize=15, autopct='%1.1f%%')
plt.ylabel('')
plt.title("Top 5 States where of most police fatalities")

### Dogs killed

In [ ]:
canine = df_police.canine.value_counts() # no of dogs who died or not
canine.plot(kind='pie', fontsize=15, autopct='%1.1f%%')
plt.ylabel(" ")
plt.title("No of humans killed vs No of dogs killed")

- 263 dogs have also been killed in the line of duty at police

### Tragic dates for police

In [ ]:
most_tragic = df_police.date.value_counts()[:10]
most_tragic.plot(kind='bar')

- 9/11 bomb attacks was one of the most disastorous days for US Police department. On a single day nearly 73 police officers died.
- 2nd most disastorous event was in 27th April, 1870 when structure collapsed and almost 11 police men died

### Police deaths over the years

In [ ]:
# thanks to jpmillers notebook
killed = df_police.loc[df_police.canine==False]
killed.groupby("year")["person"].size().plot.line()

- More police deaths happend during 1920 to 1930s

## Feature Engineering

In [ ]:
df_police['F_job_pos'] = df_police['person'].apply(lambda x: str(x).partition(' ')[0].strip())

In [ ]:
df_police['Remain_job_position'] = df_police['person'].apply(lambda x: str(x).partition(' ')[2].strip())


In [ ]:
display(df_police['F_job_pos'].value_counts()[:14])

- It's evident from the data, most of the data points are having same positions and lack consistent spellings.
- PatrolMan are killed the most

## Feature Engineering date

In [ ]:
# convert string to datetime64
df_police['date'] = pd.to_datetime(df_police['date'])

df_police.loc[:, "year"] = df_police["date"].dt.year
df_police.loc[:, "weekofyear"] = df_police["date"].dt.weekofyear
df_police.loc[:, "month"] = df_police["date"].dt.month
df_police.loc[:, "dayofweek"] = df_police["date"].dt.dayofweek
df_police.loc[:, "day"] = df_police["date"].dt.day
df_police.loc[:, "weekend"] = (df_police.date.dt.weekday >=5).astype(int)

In [ ]:
print(f"The available data is between range {df_police['date'].min()} and {df_police['date'].max()}")

## Trends of deaths

In [ ]:
df_police['count'] = 1

In [ ]:
df_1950 = df_police[(df_police['year']>=1950) &(df_police['year']<=2016)]
dfc = df_1950.copy()
dfc['count'] = df_1950['count'].cumsum()
dfc = pd.pivot_table(dfc, index='year', values='count')

bcr.bar_chart_race(df = dfc, title = "Cumulative police deaths from 1950-2010", figsize=(5, 3),cmap = ((["red"] * 7)))

## States where most number of Police men are martyered

In [ ]:
state_count_policekillings = df_police[['state', 'person']].groupby(["state",]).count().reset_index()
state_count_policekillings.head()
state_count_policekillings.columns = ['State', 'Police_killed']


In [ ]:
state_count_policekillings.sort_values(by='Police_killed', ascending=False)[:20]

## Detailed look into causes with NLPProfiler Library

- Don't run the entire dataset w/ NLP Profiler library

In [ ]:
text_dataframe = pd.DataFrame(df_ml, columns=['sentence'])
text_dataframe.head()

In [ ]:
%%time
profiled_text_dataframe = apply_text_profiling(df_ml, 'sentence')
profiled_text_dataframe.head()

In [ ]:
profiled_text_dataframe

In [ ]:
profiled_text_dataframe['sentiment_polarity'].hist(xlabelsize=15, ylabelsize=15)

## US State map where most police forces killed per state

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_count_policekillings,
    columns=['State', 'Police_killed'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Police forces killed per state'
).add_to(m)

folium.LayerControl().add_to(m)

m

- Everything in that snippet of code just works and follows things mentioned in [Folium docs](https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps)

> Then why are we getting a black map, which doesn't say anything?

In [ ]:
print(state_count_policekillings.shape)

- Let me plug the example mentioned in the getting started guide of Folium

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
state_count_policekillings.shape

In [ ]:
state_data.shape

**Oh things are terribly going wrong, There are only 50 states in United States of America while the data of police martyred consists of 60 states.**

Let's go ahead and drop these states

In [ ]:
state_count_policekillings.State = state_data.State
state_count_policekillings = state_count_policekillings.dropna()
print(state_count_policekillings.shape)
state_count_policekillings.head()

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_count_policekillings,
    columns=['State', 'Police_killed'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Police forces killed per state'
).add_to(m)

folium.LayerControl().add_to(m)

m

Great now we can see our US map representing the states where most amount of police where martyred! Finding this bug took me 2 days though :)

## More work coming soon